# Parsing csv to xlsx
## Data Sources
https://api.ehealth.gov.ua/api/services
## Commit git
- 3ed8a2e Start
- 2550448b First read json


In [18]:
import pandas as pd
import re

df = pd.read_json('./data/processed/services.json', 
                  orient='records')
# конвертація datetime64[ns, UTC] в object ('2020-03-31') без часу
for i in ['inserted_at', 'updated_at']:
    df[i] = pd.to_datetime(df[i]).dt.date

In [20]:
df.to_excel("./data/interim/first.xlsx", index=False)

## Считування за допомогою модуля json

In [22]:
import json
# считування json файла в пітоновський словник type(data) -> dict
with open('./data/raw/services.json', 'r', encoding='UTF-8') as file:
    dct = json.load(file)